In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
# Load GOT_AMENITY_EDUCATION_CHILD, GOT_AMENITY_POINTS, OSM_AMENITY_SHOPPING_GROCERY
# Load the data into a DataFrame

gdf_amenity_education_child = gpd.read_file("data/amenity/GOT_AMENITY_EDUCATION_CHILD.shp")
gdf_amenity_points = gpd.read_file("data/amenity/OSM_AMENITY_POINTS.shp")
gdf_amenity_shopping_grocery = gpd.read_file("data/amenity/OSM_AMENITY_SHOPPING_GROCERY.shp")


gdf_amenity_education_child.drop(columns=["latitude", "longitude"], inplace=True)

# Mapping from original column names to new descriptive names
location_types = {
    'E_for': 'EDUCATION_förskola',
    'E_forskls': 'EDUCATION_förskoleklass',
    'E_grund': 'EDUCATION_grundskola',
    'E_gym': 'EDUCATION_gymnasieskola',
    'E_fritids': 'EDUCATION_fritidshem'
}

# Rename the columns based on the above mapping
gdf_amenity_education_child.rename(columns=location_types, inplace=True)

# Extract the column names from the location_types mapping
location_names = list(location_types.values())

# Handle rows with multiple activities
expanded_rows = []
for idx, row in gdf_amenity_education_child.iterrows():
    active_columns = row[location_names][row[location_names] == 1].index
    for activity in active_columns:
        new_row = row.copy()
        new_row['activity'] = activity
        expanded_rows.append(new_row)

# Create a new geodataframe from the expanded rows
gdf_amenity_education_child_expanded = gpd.GeoDataFrame(expanded_rows, crs=gdf_amenity_education_child.crs)
# Set the geometry to the point
gdf_amenity_education_child_expanded.set_geometry("geometry", inplace=True)

# Drop the original activity columns
gdf_amenity_education_child_expanded.drop(columns=location_names, inplace=True)

# Rename the 'address' column to 'amenity'
gdf_amenity_education_child_expanded.rename(columns={"address": "amenity"}, inplace=True)

# Add the 'area' column
gdf_amenity_education_child_expanded["area"] = 0
gdf_amenity_points["area"] = 0

# Tansform all the crs to EPSG:3006 if not already
gdf_amenity_education_child_expanded = gdf_amenity_education_child_expanded.to_crs(epsg=3006)
gdf_amenity_points = gdf_amenity_points.to_crs(epsg=3006)
gdf_amenity_shopping_grocery = gdf_amenity_shopping_grocery.to_crs(epsg=3006)


# Merge the dataframes
merged_gdf = gpd.GeoDataFrame(pd.concat([gdf_amenity_education_child_expanded, gdf_amenity_points, gdf_amenity_shopping_grocery], ignore_index=True), crs=gdf_amenity_education_child.crs)
# Save the merged dataframe to a shapefile in data/processed as ALL_AMENITIES.shp
merged_gdf.to_file("data/processed/ALL_AMENITIES.shp")





In [1]:
# Adding playgrounds to ALL AMENITIES
import geopandas as gpd
# load ALL_AMENITIES
all_amenities = gpd.read_file("../data/processed/ALL_AMENITIES.shp")

In [6]:
playgrounds = gpd.read_file("data/scraped/GOT_AMENITY_LEISURE_PLAYGROUND.shp")

In [7]:
all_amenities.head()

,name,amenity,activity,area,geometry
0,ABC / All about children,Värmlandsgatan 19,EDUCATION_förskola,0.0,POINT (318031.044 6399237.244)
1,Albotorget 5 förskola,Albotorget 5,EDUCATION_förskola,0.0,POINT (318739.516 6398568.599)
2,Algebraförskolan,Missionshusvägen 1,EDUCATION_förskola,0.0,POINT (327547.831 6408953.546)
3,Allmänna Vägen 40 förskola,Allmänna Vägen 40C,EDUCATION_förskola,0.0,POINT (316807.298 6398977.791)
4,Amhults Byväg 10 förskola,Amhults Byväg 10,EDUCATION_förskola,0.0,POINT (307684.060 6401177.388)


In [8]:
playgrounds.head()

,name,amenity,activity,area,geometry
0,Akkas gata (lekplats 512),NaN,LEISURE_playground,0,POINT (320724.119 6405255.484)
1,Amiralitetsgatan (lekplats 100),NaN,LEISURE_playground,0,POINT (316656.009 6398900.245)
2,Angeredsparken (lekplats 508),NaN,LEISURE_playground,0,POINT (324998.551 6409795.474)
3,Annikas Gata (lekplats 5),NaN,LEISURE_playground,0,POINT (314421.911 6391194.399)
4,Arbogagatan (lekplats 123),NaN,LEISURE_playground,0,POINT (316167.396 6401563.515)


In [10]:
import pandas as pd
merged_gdf = gpd.GeoDataFrame(pd.concat([all_amenities, playgrounds], ignore_index=True), crs=all_amenities.crs)

In [11]:
# Save to "../data/processed/ALL_AMENITIES.shp"
merged_gdf.to_file("../data/processed/ALL_AMENITIES.shp")